In [ ]:
import pandas as pd
from collections import defaultdict
import json
import itertools

In [ ]:
df = pd.read_csv('raw_sen.csv', index_col=[0])

In [ ]:
stop_file = 'stopwords.txt'

with open(stop_file, 'r') as infile:
    stop_words = [word.strip() for word in infile.readlines()]

def sort_dict(d):
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}

def filter_tweet(tweet):
    word_list = []
    words = [word.strip().lower() for word in tweet.split()]
    words_filtered = [word for word in words if word not in stop_words]
    words_no_http = [word for word in words_filtered if not word.startswith('http')]
    for word in words_no_http:
        if len(word) > 2 and word != 'rt' and not word.startswith('&amp'):
            word_list.append(word)
    return word_list

def create_dict(df, handles):
    
    big_dict = {}
    #currently retweets are treated as tweets from the senator
    for handle in handles: 
        handle_dict = defaultdict(int)
        for tweet in df[df['handle'] == handle]['tweet']:
            words = [w for w in filter_tweet(tweet)]
            for word in words:
                handle_dict[word] += 1
        big_dict[handle] = sort_dict(handle_dict)
    return big_dict  

In [ ]:
df['tweet'][100]

In [ ]:
print(filter_tweet(df['tweet'][100]))

In [ ]:
sen_dict = create_dict(df, df['handle'].unique())

In [ ]:
with open('sen_words.json', 'w') as fp:
    json.dump(sen_dict, fp)

In [ ]:
def combine_dicts(d):
    t_dict = defaultdict(int)
    for key in d.keys():
        for word in d[key].keys():
            t_dict[word] += d[key][word]
    return t_dict

def dict_word_limit(d, max_words):
    import itertools
    s_dict = sort_dict(d)
    return sort_dict(dict(itertools.islice(s_dict.items(), max_words)))

In [ ]:
sen_dict_combined = combine_dicts(sen_dict)

In [ ]:
for num in [50,200,300,500,1000,2000,3000,5000,10000]:
    sen_words_temp = dict_word_limit(sen_dict_combined, num)
    
    with open(f"sen_words_{num}.json", 'w') as fp:
        json.dump(sen_words_temp, fp)